In [1]:
!pipenv --version

pipenv, version 2024.0.1


In [2]:
import json

def get_first_hash(pipfile_lock_path, library_name):
    # Load the Pipfile.lock content
    with open(pipfile_lock_path, 'r') as file:
        pipfile_lock = json.load(file)
    
    # Check in both "default" and "develop" sections for the library
    for section in ["default", "develop"]:
        if library_name in pipfile_lock.get(section, {}):
            # Get the hashes associated with the library
            hash_info = pipfile_lock[section][library_name].get("hashes", [])
            # Return the first hash if available
            return hash_info[0] if hash_info else None
    
    # Return None if the library or hash is not found
    return None

# Example usage
pipfile_lock_path = 'Pipfile.lock'
library_name = 'scikit-learn'  # Replace with the desired library name
first_hash = get_first_hash(pipfile_lock_path, library_name)
print(f"A2:The first hash of '{library_name}' is: {first_hash}")


A2:The first hash of 'scikit-learn' is: sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445


In [5]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

# Load the DictVectorizer and the model
with open('dv.bin', 'rb') as dv_file:
    dv = pickle.load(dv_file)

with open('model1.bin', 'rb') as model_file:
    model = pickle.load(model_file)

# Define the client data
client_data = {"job": "management", "duration": 400, "poutcome": "success"}

# Transform the client data using the DictVectorizer
X = dv.transform([client_data])

# Predict the probability for class 1 (the client will get a credit)
probability = model.predict_proba(X)[0, 1]

# Print the result
print(f"A3 The probability that this client will get a credit is: {probability:.3f}")

A3 The probability that this client will get a credit is: 0.759


In [9]:
import pickle
from flask import Flask, request, jsonify
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import threading

# Load the DictVectorizer and the model
with open('dv.bin', 'rb') as dv_file:
    dv = pickle.load(dv_file)

with open('model1.bin', 'rb') as model_file:
    model = pickle.load(model_file)

# Initialize Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    client_data = request.get_json()
    # Transform client data
    X = dv.transform([client_data])
    # Predict probability
    probability = model.predict_proba(X)[0, 1]
    # Return the probability as JSON
    return jsonify({'credit_probability': probability})

# Function to run the Flask app in a thread
def run_app():
    app.run(host='0.0.0.0', port=9696, debug=False, use_reloader=False)

# Start Flask in a background thread
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:9696
 * Running on http://192.168.1.108:9696
Press CTRL+C to quit
127.0.0.1 - - [31/Oct/2024 00:29:28] "POST /predict HTTP/1.1" 200 -


In [10]:
import requests
url = "http://127.0.0.1:9696/predict"
client = {"job": "student", "duration": 280, "poutcome": "failure"}
response = requests.post(url, json=client).json()

print(f"A4 The probability that the client will get a credit is {round(response['credit_probability'],3)}")

A4 The probability that the client will get a credit is 0.335


In [3]:
!docker images

REPOSITORY                                      TAG            IMAGE ID       CREATED             SIZE
homework05                                      latest         0354cc7fa249   2 minutes ago       504MB
<none>                                          <none>         55073cd9225b   4 minutes ago       504MB
<none>                                          <none>         adad1d334b21   About an hour ago   495MB
svizor/zoomcamp-model                           3.11.5-slim    975e7bdca086   12 days ago         130MB
primalhealth-flask_app                          latest         148f31e5a077   2 weeks ago         2.08GB
redis                                           7.2-bookworm   f4b3e8ad90fe   3 weeks ago         116MB
llm_project_llama-streamlit                     latest         fc3cd2231e5a   4 weeks ago         9.6GB
dpage/pgadmin4                                  latest         a67d330eef3b   5 weeks ago         485MB
docker.elastic.co/elasticsearch/elasticsearch   8.15.1         8

In [2]:
import requests
url = "http://127.0.0.1:9696/predict"
client = {"job": "management", "duration": 400, "poutcome": "success"}
response = requests.post(url, json=client).json()

print(f"A4 The probability that the client will get a credit is {round(response['credit_probability'],3)}")

A4 The probability that the client will get a credit is 0.759
